In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.externals import joblib
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import KFold
from sklearn import metrics
import scipy as sp
from sklearn.grid_search import RandomizedSearchCV
from sklearn.metrics import roc_auc_score

advertiserlist_path = '../data/adtech_intern_advertiserlist.csv'
log_path = '../data/intern_samplelog.csv'

In [2]:
advertiserlist = pd.read_csv(advertiserlist_path)
log_original = pd.read_csv(log_path)
log = log_original

In [3]:
log.loc[:,'os'][log.os=='iOS'] = 0
log.loc[:,'os'][log.os=='Android'] = 1

/Users/kei/penv2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/kei/penv2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [4]:
def hashnum(modnum, data):
    mod = 10 ** modnum
    return hash(data) % mod

In [17]:
ad_num = 10
logEachAd = [log[log.advertiser==ad].drop('advertiser', axis=1)for ad in xrange(1, ad_num + 1)]
input_type='string'
param_distributions={'learning_rate': [0.1],
                     'max_depth': sp.stats.randint(1,11),}
                     # 'subsample': sp.stats.uniform(0.5,0.5),
                     # 'colsample_bytree': sp.stats.uniform(0.5,0.5),
                    # "min_child_weight" :  sp.stats.uniform(1, 10),
                    # 'gamma' : sp.stats.uniform(0, 1),
                    # "colsample_bytree" : sp.stats.uniform(0.5, 1)}

num_round = 2
modnum = 3
n_folds = 10

xgb_models = [0] * 10
aucs = [0] * 10

In [18]:
param = {'eta':1, 'silent':1, 'objective':'binary:logistic',
         'max_depth':9, 'subsample': 0.5,'colsample_bytree': 0.5,
         "min_child_weight" : 1, 'gamma' : 0, "colsample_bytree" : 0.5}
for ind, eachlog in enumerate(logEachAd):
    print '----------------- mod 10 ** %d  Advertiser %d -------------------' %(modnum, ind+1)
    log_data = eachlog.drop('click', axis=1).fillna(0)
    log_target = eachlog.click
    hashlog_data = [[hashnum(modnum, num) for num in sample] for sample in log_data.drop(['floor_price','os'], axis=1).values]
    log_data = np.c_[hashlog_data,log_data.os, log_data.floor_price]
    dlog = xgb.DMatrix(log_data,label=log_target)
    aucs[ind] = xgb.cv(param, dlog, num_round, nfold=n_folds, metrics={'auc'}, seed = 0)['test-auc-mean'].mean()
    print aucs[ind].mean()
print param
print np.mean(aucs)
    # xgb_model = xgb.train(param, dlog, num_round)
    # model_path = '../models/Ad%d_xgb.pkl'%(ind+1)
    # joblib.dump(xgb_model, model_path)

----------------- mod 10 ** 3  Advertiser 1 -------------------
0.59601645
----------------- mod 10 ** 3  Advertiser 2 -------------------
0.54192745
----------------- mod 10 ** 3  Advertiser 3 -------------------
0.600247
----------------- mod 10 ** 3  Advertiser 4 -------------------
0.54961005
----------------- mod 10 ** 3  Advertiser 5 -------------------
0.54580195
----------------- mod 10 ** 3  Advertiser 6 -------------------
0.5390913
----------------- mod 10 ** 3  Advertiser 7 -------------------
0.59278085
----------------- mod 10 ** 3  Advertiser 8 -------------------
0.5775582
----------------- mod 10 ** 3  Advertiser 9 -------------------
0.60163455
----------------- mod 10 ** 3  Advertiser 10 -------------------
0.72033325
{'subsample': 0.5, 'eta': 1, 'colsample_bytree': 0.5, 'silent': 1, 'objective': 'binary:logistic', 'gamma': 0, 'max_depth': 9, 'min_child_weight': 1}
0.586500105


In [20]:
param = {'silent':1, 'objective':'binary:logistic',
     'max_depth':3, 'subsample': 1,"min_child_weight" : 1, 'gamma' : 0, "colsample_bytree" : 1}
for ind, eachlog in enumerate(logEachAd):
    print '----------------- mod 10 ** %d  Advertiser %d -------------------' %(modnum, ind+1)
    log_data = eachlog.drop('click', axis=1).fillna(0)
    log_target = eachlog.click
    hashlog_data = [[hashnum(modnum, num) for num in sample] for sample in log_data.drop(['floor_price','os'], axis=1).values]
    log_data = np.c_[hashlog_data,log_data.os, log_data.floor_price]
    dlog = xgb.DMatrix(log_data,label=log_target)
    aucs[ind] = xgb.cv(param, dlog, num_round, nfold=n_folds, metrics={'auc'}, seed = 0)['test-auc-mean'].mean()
    print aucs[ind].mean()
print param
print np.mean(aucs)

----------------- mod 10 ** 3  Advertiser 1 -------------------
0.60618805
----------------- mod 10 ** 3  Advertiser 2 -------------------
0.53056865
----------------- mod 10 ** 3  Advertiser 3 -------------------
0.586842
----------------- mod 10 ** 3  Advertiser 4 -------------------
0.54101615
----------------- mod 10 ** 3  Advertiser 5 -------------------
0.56508595
----------------- mod 10 ** 3  Advertiser 6 -------------------
0.52313635
----------------- mod 10 ** 3  Advertiser 7 -------------------
0.5817602
----------------- mod 10 ** 3  Advertiser 8 -------------------
0.57262495
----------------- mod 10 ** 3  Advertiser 9 -------------------
0.63359255
----------------- mod 10 ** 3  Advertiser 10 -------------------
0.6807505
{'colsample_bytree': 1, 'silent': 1, 'min_child_weight': 1, 'subsample': 1, 'objective': 'binary:logistic', 'max_depth': 3, 'gamma': 0}
0.582156535


In [21]:
param = {'silent':1, 'objective':'binary:logistic',
     'max_depth':9, 'subsample': 1,"min_child_weight" : 1, 'gamma' : 0, "colsample_bytree" : 1}
for ind, eachlog in enumerate(logEachAd):
    print '----------------- mod 10 ** %d  Advertiser %d -------------------' %(modnum, ind+1)
    log_data = eachlog.drop('click', axis=1).fillna(0)
    log_target = eachlog.click
    hashlog_data = [[hashnum(modnum, num) for num in sample] for sample in log_data.drop(['floor_price','os'], axis=1).values]
    log_data = np.c_[hashlog_data,log_data.os, log_data.floor_price]
    dlog = xgb.DMatrix(log_data,label=log_target)
    aucs[ind] = xgb.cv(param, dlog, num_round, nfold=n_folds, metrics={'auc'}, seed = 0)['test-auc-mean'].mean()
    print aucs[ind].mean()
print param
print np.mean(aucs)

----------------- mod 10 ** 3  Advertiser 1 -------------------
0.66970785
----------------- mod 10 ** 3  Advertiser 2 -------------------
0.5703287
----------------- mod 10 ** 3  Advertiser 3 -------------------
0.6336613
----------------- mod 10 ** 3  Advertiser 4 -------------------
0.5824795
----------------- mod 10 ** 3  Advertiser 5 -------------------
0.59906095
----------------- mod 10 ** 3  Advertiser 6 -------------------
0.55654665
----------------- mod 10 ** 3  Advertiser 7 -------------------
0.6645466
----------------- mod 10 ** 3  Advertiser 8 -------------------
0.6675532
----------------- mod 10 ** 3  Advertiser 9 -------------------
0.71749495
----------------- mod 10 ** 3  Advertiser 10 -------------------
0.79357065
{'colsample_bytree': 1, 'silent': 1, 'min_child_weight': 1, 'subsample': 1, 'objective': 'binary:logistic', 'max_depth': 9, 'gamma': 0}
0.645495035


In [26]:
param = {'silent':1, 'objective':'binary:logistic','max_delta_step' = 0,
     'max_depth':11, 'subsample': 1,"min_child_weight" : 1, 'gamma' : 0, "colsample_bytree" : 1}
for ind, eachlog in enumerate(logEachAd):
    print '----------------- mod 10 ** %d  Advertiser %d -------------------' %(modnum, ind+1)
    log_data = eachlog.drop('click', axis=1).fillna(0)
    log_target = eachlog.click
    hashlog_data = [[hashnum(modnum, num) for num in sample] for sample in log_data.drop(['floor_price','os'], axis=1).values]
    log_data = np.c_[hashlog_data,log_data.os, log_data.floor_price]
    dlog = xgb.DMatrix(log_data,label=log_target)
    aucs[ind] = xgb.cv(param, dlog, num_round, nfold=n_folds, metrics={'auc'}, seed = 0)[['test-auc-mean','train-auc-mean']].mean()
    print aucs[ind]['test-auc-mean'], aucs[ind]['train-auc-mean']
    xgb_model = xgb.train(param, dlog, num_round)
    model_path = '../models/1st_Ad%d_xgb.pkl'%(ind+1)
    joblib.dump(xgb_model, model_path)
print param
print np.mean(np.array(aucs).T, axis=1)

----------------- mod 10 ** 3  Advertiser 1 -------------------
0.6701375 0.6802091
----------------- mod 10 ** 3  Advertiser 2 -------------------
0.5729127 0.59058545
----------------- mod 10 ** 3  Advertiser 3 -------------------
0.6369728 0.65158765
----------------- mod 10 ** 3  Advertiser 4 -------------------
0.58541265 0.60729765
----------------- mod 10 ** 3  Advertiser 5 -------------------
0.59966005 0.6203432
----------------- mod 10 ** 3  Advertiser 6 -------------------
0.5576448 0.57913425
----------------- mod 10 ** 3  Advertiser 7 -------------------
0.67648425 0.6888917
----------------- mod 10 ** 3  Advertiser 8 -------------------
0.6650497 0.6828393
----------------- mod 10 ** 3  Advertiser 9 -------------------
0.72100465 0.73465175
----------------- mod 10 ** 3  Advertiser 10 -------------------
0.8018271 0.8107741
{'colsample_bytree': 1, 'silent': 1, 'min_child_weight': 1, 'subsample': 1, 'objective': 'binary:logistic', 'max_depth': 11, 'gamma': 0}
[ 0.64871062 

In [31]:
param = {'silent':1, 'objective':'binary:logistic','max_delta_step' : 0,
     'max_depth':12, 'subsample': 1,"min_child_weight" : 1, 'gamma' : 0, "colsample_bytree" : 1}
for ind, eachlog in enumerate(logEachAd):
    print '----------------- mod 10 ** %d  Advertiser %d -------------------' %(modnum, ind+1)
    log_data = eachlog.drop('click', axis=1).fillna(0)
    log_target = eachlog.click
    hashlog_data = [[hashnum(modnum, num) for num in sample] for sample in log_data.drop(['floor_price','os'], axis=1).values]
    log_data = np.c_[hashlog_data,log_data.os, log_data.floor_price]
    dlog = xgb.DMatrix(log_data,label=log_target)
    aucs[ind] = xgb.cv(param, dlog, num_round, nfold=n_folds, metrics={'auc'}, seed = 0)[['test-auc-mean','train-auc-mean']].mean()
    print aucs[ind]['test-auc-mean'], aucs[ind]['train-auc-mean']
print param
print np.mean(np.array(aucs).T, axis=1)

----------------- mod 10 ** 3  Advertiser 1 -------------------
0.66991785 0.68124915
----------------- mod 10 ** 3  Advertiser 2 -------------------
0.57186485 0.594369
----------------- mod 10 ** 3  Advertiser 3 -------------------
0.63721185 0.6559841
----------------- mod 10 ** 3  Advertiser 4 -------------------
0.5866718 0.6123963
----------------- mod 10 ** 3  Advertiser 5 -------------------
0.6010395 0.62881865
----------------- mod 10 ** 3  Advertiser 6 -------------------
0.55713405 0.58620975
----------------- mod 10 ** 3  Advertiser 7 -------------------
0.6753738 0.69292495
----------------- mod 10 ** 3  Advertiser 8 -------------------
0.6627001 0.68771555
----------------- mod 10 ** 3  Advertiser 9 -------------------
0.7204414 0.73879895
----------------- mod 10 ** 3  Advertiser 10 -------------------
0.7990897 0.81151
{'subsample': 1, 'gamma': 0, 'colsample_bytree': 1, 'silent': 1, 'objective': 'binary:logistic', 'max_delta_step': 0, 'max_depth': 12, 'min_child_weight

In [30]:
param = {'silent':1, 'objective':'binary:logistic','max_delta_step' : 0,
     'max_depth':13, 'subsample': 1,"min_child_weight" : 1, 'gamma' : 0, "colsample_bytree" : 1}
for ind, eachlog in enumerate(logEachAd):
    print '----------------- mod 10 ** %d  Advertiser %d -------------------' %(modnum, ind+1)
    log_data = eachlog.drop('click', axis=1).fillna(0)
    log_target = eachlog.click
    hashlog_data = [[hashnum(modnum, num) for num in sample] for sample in log_data.drop(['floor_price','os'], axis=1).values]
    log_data = np.c_[hashlog_data,log_data.os, log_data.floor_price]
    dlog = xgb.DMatrix(log_data,label=log_target)
    aucs[ind] = xgb.cv(param, dlog, num_round, nfold=n_folds, metrics={'auc'}, seed = 0)[['test-auc-mean','train-auc-mean']].mean()
    print aucs[ind]['test-auc-mean'], aucs[ind]['train-auc-mean']
print param
print np.mean(np.array(aucs).T, axis=1)

----------------- mod 10 ** 3  Advertiser 1 -------------------
0.66959385 0.68207905
----------------- mod 10 ** 3  Advertiser 2 -------------------
0.571059 0.5973538
----------------- mod 10 ** 3  Advertiser 3 -------------------
0.63588375 0.6598904
----------------- mod 10 ** 3  Advertiser 4 -------------------
0.5866265 0.61941145
----------------- mod 10 ** 3  Advertiser 5 -------------------
0.59821815 0.6331994
----------------- mod 10 ** 3  Advertiser 6 -------------------
0.5566885 0.594436
----------------- mod 10 ** 3  Advertiser 7 -------------------
0.67793595 0.70079575
----------------- mod 10 ** 3  Advertiser 8 -------------------
0.66010855 0.6935021
----------------- mod 10 ** 3  Advertiser 9 -------------------
0.7186258 0.74262515
----------------- mod 10 ** 3  Advertiser 10 -------------------
0.80193805 0.81779615
{'subsample': 1, 'gamma': 0, 'colsample_bytree': 1, 'silent': 1, 'objective': 'binary:logistic', 'max_delta_step': 0, 'max_depth': 13, 'min_child_weig

In [15]:
for ind, eachlog in enumerate(logEachAd):
    print '----------------- Advertiser %d -------------------' %(ind+1)
    log_data = eachlog.drop('click', axis=1)
    log_target = eachlog.click.values
    hashlog_data = [[hashnum(modnum, num) for num in sample] for sample in log_data.drop(['floor_price','os'], axis=1).values]
    log_data = np.c_[hashlog_data,log_data.os]
    sample_length = len(log_target)/10
    trainX = log_data[:-sample_length]
    trainY = log_target[:-sample_length]
    testX = log_data[-sample_length:]
    testY = log_target[-sample_length:]
    # dlog = xgb.DMatrix(log_data, label=log_target)
    xgb_model = xgb.XGBClassifier(param)
    rs = RandomizedSearchCV(xgb_model, param_distributions, cv=10, n_iter=1, scoring="roc_auc", verbose=2, n_jobs = 1, random_state=0 )
    rs.fit(trainX,trainY)
    predict = rs.predict(testX)
    print roc_auc_score(testY, predict)
    best_param = {'learning_rate': rs.best_estimator_.learning_rate,
              'max_depth': rs.best_estimator_.max_depth,
                 'subsample': rs.best_estimator_.subsample,
                  "min_child_weight" :  rs.best_estimator_.min_child_weight,
              'gamma': rs.best_estimator_.gamma,
              'silent': rs.best_estimator_.silent,
              'objective': rs.best_estimator_.objective,
                  "colsample_bytree" : rs.best_estimator_.colsample_bytree
              }
    dtrain = xgb.DMatrix(log_data, label=log_target)
    bst = xgb.train(best_param, dtrain, num_round)
    model_path = '../models/Ad%d_xgb.pkl'%(ind+1)
    joblib.dump(bst, model_path)

----------------- Advertiser 1 -------------------
Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV] learning_rate=0.1, max_depth=5 ..................................
[CV] ......................... learning_rate=0.1, max_depth=5 -   2.7s
[CV] learning_rate=0.1, max_depth=5 ..................................
[CV] ......................... learning_rate=0.1, max_depth=5 -   2.7s
[CV] learning_rate=0.1, max_depth=5 ..................................
[CV] ......................... learning_rate=0.1, max_depth=5 -   2.7s
[CV] learning_rate=0.1, max_depth=5 ..................................
[CV] ......................... learning_rate=0.1, max_depth=5 -   2.7s
[CV] learning_rate=0.1, max_depth=5 ..................................
[CV] ......................... learning_rate=0.1, max_depth=5 -   2.7s
[CV] learning_rate=0.1, max_depth=5 ..................................
[CV] ......................... learning_rate=0.1, max_depth=5 -   2.7s
[CV] learning_rate=0.1, max_depth=5 

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   27.5s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   53.0s finished



0.500493248106
----------------- Advertiser 3 -------------------
Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV] learning_rate=0.1, max_depth=2 ..................................
[CV] ......................... learning_rate=0.1, max_depth=2 -   1.4s
[CV] learning_rate=0.1, max_depth=2 ..................................
[CV] ......................... learning_rate=0.1, max_depth=2 -   2.1s
[CV] learning_rate=0.1, max_depth=2 ..................................
[CV] ......................... learning_rate=0.1, max_depth=2 -   1.7s
[CV] learning_rate=0.1, max_depth=2 ..................................
[CV] ......................... learning_rate=0.1, max_depth=2 -   1.5s
[CV] learning_rate=0.1, max_depth=2 ..................................
[CV] ......................... learning_rate=0.1, max_depth=2 -   1.4s
[CV] learning_rate=0.1, max_depth=2 ..................................
[CV] ......................... learning_rate=0.1, max_depth=2 -   1.6s
[CV] learning_rate=0

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   15.5s finished


KeyboardInterrupt: 

In [ ]:
rs.fit(trainX,trainY)
predict = rs.predict(testX)

In [ ]:
for ind, eachlog in enumerate(logEachAd):
    print '----------------- Advertiser %d -------------------' %(ind+1)
    log_data = eachlog.drop('click', axis=1)
    log_target = eachlog.click.values
    hashlog_data = [[hashnum(modnum, num) for num in sample] for sample in log_data.drop(['floor_price','os'], axis=1).values]
    log_data = np.c_[hashlog_data,log_data.os]
    dlog = xgb.DMatrix(log_data, label=log_target)
    xgb_model = xgb.train(param, dlog_train, num_round)
    
    kf = KFold(len(log_target), n_folds=n_folds, shuffle=True, random_state=0)
    for train_index, test_index in kf:
        dlog_train = xgb.DMatrix(
            log_data[train_index], label=log_target[train_index])
        dlog_test = xgb.DMatrix(
            log_data[test_index], label=log_target[test_index])
        xgb_model = xgb.train(param, dlog_train, num_round)
        predict = xgb_model.predict(dlog_test)
        # print metrics.roc_auc_score(log_target[test_index], predict)
    xgb_models[ind] = xgb.train(param, dlog, num_round)